Data cleaning, analysis and visualization in Python using Jupyter Notebooks.

Importing the necessary libraries.


In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

Acquiring the data from the CSV file.

In [7]:
netflix = pd.read_csv('netflix_titles.csv')

The data at first glance

In [8]:
#First 5 rows of the dataframe
netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [9]:
#Number of rows and columns
netflix.shape

(8807, 12)

In [11]:
#Statistical summary of our data
netflix.describe()

,release_year
count,8807.000000
mean,2014.180198
std,8.819312
min,1925.000000
25%,2013.000000
50%,2017.000000
75%,2019.000000
max,2021.000000


In [10]:
netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


Now that our data has been imported correctly and we've taken a look at it, we can start noticing a few problems which may skew our analysis process:

1.Null values are present in a few columns. Let's get the percentage of null values for each column:

In [28]:
percentages = netflix.isnull().mean()*100
null_columns = round(percentages[percentages > 0], 2)
null_columns.sort_values(ascending=False, inplace=True)
null_columns

director      29.91
country        9.44
cast           9.37
date_added     0.11
rating         0.05
duration       0.03
dtype: float64

We can clearly tell there will be a problem when performing our analysis since more than a quarted of the "director" column is filled with null values, and almost 10% of the "country" and "cast" columns are null as well.

To fix this, we will fill these columns with an empty string value (""), since this will not affect our charts and plots.

However, we can see a small percentage of missing values in the "date_added", "duration" and "rating" columns as well. Since these will make a large part of our plots, we shall remove the rows that containg null values in these columns in order to not skew with our analysis.

In [31]:
#Filling in the null values
netflix[['director', 'country', 'cast']].dtypes

director    object
country     object
cast        object
dtype: object

In [127]:
netflix[['director', 'country', 'cast']] = netflix[['director', 'country', 'cast']].fillna("")
netflix.head()

,show_id,type,title,director,cast,country,date_added,year_added,month_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,,United States,"September 25, 2021",2021,9,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,9,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",,"September 24, 2021",2021,9,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,,,,"September 24, 2021",2021,9,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,9,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


Now that we have filled in the missing values for these columns, we need to do something about the rest. Since filling them as well would skew our plots and there is a very small amount of data missing, we will drop the rows that contain missing data in the "date_added", "duration" and "rating" columns.

In [128]:
netflix.isnull().sum()

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
year_added      0
month_added     0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64

Since these are the only columns that still have null values, we can drop them as follows:

In [129]:
netflix.dropna(inplace=True)
netflix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8790 entries, 0 to 8806
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8790 non-null   object
 1   type          8790 non-null   object
 2   title         8790 non-null   object
 3   director      8790 non-null   object
 4   cast          8790 non-null   object
 5   country       8790 non-null   object
 6   date_added    8790 non-null   object
 7   year_added    8790 non-null   int64 
 8   month_added   8790 non-null   int64 
 9   release_year  8790 non-null   int64 
 10  rating        8790 non-null   object
 11  duration      8790 non-null   object
 12  listed_in     8790 non-null   object
 13  description   8790 non-null   object
dtypes: int64(3), object(11)
memory usage: 1.3+ MB


In [130]:
netflix.isnull().any()

show_id         False
type            False
title           False
director        False
cast            False
country         False
date_added      False
year_added      False
month_added     False
release_year    False
rating          False
duration        False
listed_in       False
description     False
dtype: bool

Now that our data is free of null values, we should start cleaning the not-null values.

In [50]:
netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,,,,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [126]:
netflix['date_added'].dtype

dtype('O')

Removing the trailing whitespaces from all of the string and mixed columns.

In [144]:
for i in netflix.columns:
    if netflix[i].dtype=='O':
        netflix[i] = netflix[i].str.strip()
netflix.head()

,show_id,type,title,director,cast,country,date_added,year_added,month_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,,United States,"September 25, 2021",2021,9,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,9,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",,"September 24, 2021",2021,9,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,,,,"September 24, 2021",2021,9,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,9,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


Taking a look at the "date_added" column, we can see it is of type 'object' meaning it's a string. We will separate the column into 2 distinct columns: one for the year and one for the month.

In [145]:
from datetime import datetime
years = []
months = []
for i in range(len(netflix)):
    date = netflix['date_added'].iloc[i]
    date = datetime.strptime(date, "%B %d, %Y")

    years.append(date.year)
    months.append(date.month)

if 'year_added' not in netflix.columns and 'month_added' not in netflix.columns:
    netflix.insert(7, 'year_added', years, allow_duplicates=True)
    netflix.insert(8, 'month_added', months, allow_duplicates=True)
netflix.head()

,show_id,type,title,director,cast,country,date_added,year_added,month_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,,United States,"September 25, 2021",2021,9,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,9,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",,"September 24, 2021",2021,9,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,,,,"September 24, 2021",2021,9,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,9,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


Now that our "date_added" columns has been cleaned, we need to take a look at the other columns to see which one require more cleaning and we'll start with the "type" column.


In [146]:
netflix.type.value_counts()

Movie      6126
TV Show    2664
Name: type, dtype: int64

We can see the column is formatted correctly as there can only be Movies and TV Shows as the values. We will later separate the dataframe into 2 subsets, one for each type, but for now, we have to finish the cleaning process.

We'll now take a look at the "country" column:

In [149]:
netflix['country'].value_counts()

United States                             2809
India                                      972
                                           829
United Kingdom                             418
Japan                                      243
                                          ... 
Romania, Bulgaria, Hungary                   1
Uruguay, Guatemala                           1
France, Senegal, Belgium                     1
Mexico, United States, Spain, Colombia       1
United Arab Emirates, Jordan                 1
Name: country, Length: 749, dtype: int64

In [120]:
movies = netflix.groupby('type').get_group('Movie')
tv_shows = netflix.groupby('type').get_group('TV Show')

min                                               10 min
max                                               99 min
sum    90 min91 min125 min104 min127 min91 min67 min9...
Name: duration, dtype: object